In [1]:
# Importacion de librerias a utilizar
import pandas as pd
import json
import locale
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
# Convierto mi archivo csv en un dataframe de pandas
df = pd.read_csv("movies_dataset_2do_intento.csv")

C:\Users\Yanina Lucia\AppData\Local\Temp\ipykernel_10024\3851153582.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_dataset_2do_intento.csv")


In [3]:
# Quito las columnas las cuales no tomaremos en cuenta para trabajar con ellas
df = df.drop(["adult", "homepage", "imdb_id", "original_title", "poster_path", "runtime", "video"], axis= 1)

In [4]:
# Creo esta funcion para desanidar las columnas que lo requieran
def safe_json_loads(s):
    try:
        return json.loads(s) #Cargo datos Json
    except:
        return {} #Relleno con {}

In [5]:
#Cambio las comillas simples por dobles
df["belongs_to_collection"] = df["belongs_to_collection"].str.replace("'",'"')
#Cambio los "None" por ""
df["belongs_to_collection"] = df["belongs_to_collection"].str.replace('"backdrop_path": None}', '"backdrop_path": ""}')
#Cambio "NaN" por "{}""
df["belongs_to_collection"] = df["belongs_to_collection"].fillna('{}')
#Normalizo datos Json en una tabla plana y los cargo a la base de datos
df_belongs = pd.json_normalize(df["belongs_to_collection"].apply(safe_json_loads))
#dropeo las columnas que no necesito
df_belongs = df_belongs.drop(["id", "poster_path", "backdrop_path"], axis = 1)
#Sustituyo datos
df["belongs_to_collection"] = df_belongs
#Cambiamos "NaN" por "No collection"
df["belongs_to_collection"] = df["belongs_to_collection"].fillna("No collection")

C:\Users\Yanina Lucia\AppData\Local\Temp\ipykernel_10024\1957826267.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["belongs_to_collection"] = df["belongs_to_collection"].str.replace('"backdrop_path": None}', '"backdrop_path": ""}')


HASTA ACA YA LA COLUMNA "BELONG_TO_COLLECTION" ESTA DESANIDADA, AHORA IRE CON LA DE "GENRES"

In [6]:
# Cambiamos las comillas simples por dobles
df['genres'] = df['genres'].str.replace("'", '"')
# Cambiamos los NaN por {}
df['genres'] = df['genres'].fillna('{}')
# Normalizamos los datos JSON (listas de diccionarios) en una tabla plana y los cargamos
df_genres = pd.json_normalize(df['genres'].apply(safe_json_loads))  
#Creo un lista que voy a llenar con los datos de cada columna
dfs_genres = []
# Recorremos las columnas de los datos normalizados que todavía tienen formato JSON (diccionarios)
for i in range (df_genres.shape[1]):  
    df_genres[[i]] = df_genres[[i]].astype(str)
    df_genres[i] = df_genres[i].str.replace("'", '"')
    df_genres[i] = df_genres[i].fillna('{}')
    globals()['df_genres{}'.format(i)] = pd.json_normalize(df_genres[i].apply(safe_json_loads)).astype(str)
    dfs_genres.append(globals()['df_genres{}'.format(i)])  # Agregar cada DataFrame resultante a la lista
# Concatena todos los DataFrames de la lista en uno solo
df_genres_resul = pd.concat(dfs_genres, axis=1)
#Tomo todas las columnas para trabajar con ellas
df_nuevo = df_genres_resul.iloc[:, 0:16]
#dropeo las columnas que no me sirven
df_nuevo = df_nuevo.drop("id", axis= 1)
#Ingreso los datos en cada columna
df_nuevo['nueva_col'] = df_nuevo.apply(lambda row: [x for x in row[['name']].values if not pd.isna(x)], axis=1)
#Filtro las filas y elimino los "nan"y los ingreso en una nueva columna
df_nuevo['nueva_col'] = [list(filter(lambda x: x!='nan', lst)) for lst in df_nuevo['nueva_col']]
#dropeo las columnas que ya no me sirven
df_nuevo.drop(["name"], axis = 1)
#Sustituyo la columna
df['genres'] = df_nuevo["nueva_col"]

HASTA ACA YA LA COLUMNA "GENRES" ESTA DESANIDADA, AHORA IRE CON LA DE "PRODUCTION_COMPANIES"

In [7]:
# Cambiamos las comillas simples por dobles
df['production_companies'] = df['production_companies'].str.replace("'", '"')
# Cambiamos los NaN por {}
df['production_companies'] = df['production_companies'].fillna('{}')
# Normalizamos los datos JSON (listas de diccionarios) en una tabla plana y los cargamos
df_production = pd.json_normalize(df['production_companies'].apply(safe_json_loads))
#Creo un lista que voy a llenar con los datos de cada columna
dfs_companie = []
# Recorremos las columnas de los datos normalizados que todavía tienen formato JSON (diccionarios)
for i in range (0, df_production.shape[1]):
    df_production[[i]] = df_production[[i]].astype(str)
    df_production[i] = df_production[i].str.replace("'", '"')
    df_production[i] = df_production[i].fillna('{}')
    globals()['df_production{}'.format(i)] = pd.json_normalize(df_production[i].apply(safe_json_loads)).astype(str)
    dfs_companie.append(globals()['df_production{}'.format(i)])  # Agregar cada DataFrame resultante a la lista
# Concatenar todos los DataFrames de la lista en uno solo
df_production_resul = pd.concat(dfs_companie, axis=1)
#Tomo todas las columnas para trabajar con ellas
df_nuevo = df_production_resul.iloc[:, 0:52]
#dropeo las columnas que no me sirven
df_nuevo = df_nuevo.drop("id", axis= 1)
#Ingreso los datos en cada columna
df_nuevo['nueva_col'] = df_nuevo.apply(lambda row: [x for x in row[['name']].values if not pd.isna(x)], axis=1)
#Filtro las filas y elimino los "nan"y los ingreso en una nueva columna
df_nuevo['nueva_col'] = [list(filter(lambda x: x!='nan', lst)) for lst in df_nuevo['nueva_col']]
#dropeo las columnas que ya no me sirven
df_nuevo.drop(["name"], axis = 1)
#Sustituyo la columna
df["production_companies"] = df_nuevo["nueva_col"]

HASTA ACA YA LA COLUMNA "PRODUCTION_COMPANIES" ESTA DESANIDADA, AHORA IRE CON LA DE "PRODUCTION_COUNTRIES"

In [8]:
# Cambiamos las comillas simples por dobles
df['production_countries'] = df['production_countries'].str.replace("'", '"')
# Cambiamos los NaN por {}
df['production_countries'] = df['production_countries'].fillna('{}')
# Normalizamos los datos JSON (listas de diccionarios) en una tabla plana y los cargamos
df_countries = pd.json_normalize(df['production_countries'].apply(safe_json_loads))
#Creo un lista que voy a llenar con los datos de cada columna
dfs_country = []
# Recorremos las columnas de los datos normalizados que todavía tienen formato JSON (diccionarios)
for i in range (0, df_countries.shape[1]):
    df_countries[[i]] = df_countries[[i]].astype(str)
    df_countries[i] = df_countries[i].str.replace("'", '"')
    df_countries[i] = df_countries[i].fillna('{}')
    globals()['df_countries{}'.format(i)] = pd.json_normalize(df_countries[i].apply(safe_json_loads)).astype(str)
    dfs_country.append(globals()['df_countries{}'.format(i)])  # Agregar cada DataFrame resultante a la lista
# Concatenar todos los DataFrames de la lista en uno solo
df_countries_resul = pd.concat(dfs_country, axis=1)
#Tomo todas las columnas para trabajar con ellas
df_nuevo = df_countries_resul.iloc[:, 0:50]
#dropeo las columnas que no me sirven
df_nuevo = df_nuevo.drop(["iso_3166_1"], axis = 1)
#Ingreso los datos en cada columna
df_nuevo['nueva_col'] = df_nuevo.apply(lambda row: [x for x in row[["name"]].values if not pd.isna(x)], axis=1)
#Filtro las filas y elimino los "nan"y los ingreso en una nueva columna
df_nuevo['nueva_col'] = [list(filter(lambda x: x!='nan', lst)) for lst in df_nuevo['nueva_col']]
#dropeo las columnas que ya no me sirven
df_nuevo = df_nuevo.drop(["name"], axis = 1)
#Sustituyo la columna
df["production_countries"] = df_nuevo["nueva_col"]

HASTA ACA YA LA COLUMNA "PRODUCTION_COUNTRIES" ESTA DESANIDADA, AHORA IRE CON LA DE "SPOKEN_LENGUAGES"

In [9]:
# Desanidamos los datos de la columna 'spoken_languages'
df['spoken_languages'] = df['spoken_languages'].str.replace("'", '"')  # Sustituimos las comillas simples por dobles
df['spoken_languages'] = df['spoken_languages'].fillna('{}')   # Rellenamos NaN con {}
df_spoken_languages = pd.json_normalize(df['spoken_languages'].apply(safe_json_loads))  # Normalizamos los datos JSON (lista de diccionarios) en una tabla plana y los cargamos
dfs_languages = []

for i in range (df_spoken_languages.shape[1]):  # Recorremos las columnas de los datos normalizados que todavía tienen formato JSON (diccionarios)
    df_spoken_languages[[i]] = df_spoken_languages[[i]].astype(str)  # Indicamos que los datos son tipo string
    df_spoken_languages[i] = df_spoken_languages[i].str.replace("'", '"')  # Sustituimos las comillas simples por dobles
    df_spoken_languages[i] = df_spoken_languages[i].fillna('{}')   # Rellenamos NaN con {}
    # Normalizamos los datos JSON en una tabla plana y los cargamos
    # Copiamos los valores no nulos en la lista de cada fila del DataFrame
    globals()['df_spoken_languages{}'.format(i)] = pd.json_normalize(df_spoken_languages[i].apply(safe_json_loads)).astype(str)
    dfs_languages.append(globals()['df_spoken_languages{}'.format(i)])  # Agregamos cada DataFrame resultante a la lista

df_languages_resul = pd.concat(dfs_languages, axis=1)  # Concatena todos los DataFrames de la lista en uno solo
df_nuevo = df_languages_resul.iloc[:, 0:26]  # Localizamos todas las filas para las 26 columnas del DataFrame df_languages_resul y las guardamos en df_nuevo
df_nuevo = df_nuevo.drop("iso_639_1", axis= 1)  # Dropeamos las columnas id iso_639_1
# Copiamos los valores no nulos en la lista de cada fila del DataFrame
df_nuevo['nueva_col'] = df_nuevo.apply(lambda row: [x for x in row[['name']].values if not pd.isna(x)], axis=1)
df_nuevo['nueva_col'] = [list(filter(lambda x: x!='nan', lst)) for lst in df_nuevo['nueva_col']]
df_nuevo.drop(["name"], axis = 1)  # Dropeamos las columnas name
df["spoken_languages"] = df_nuevo["nueva_col"]  # Sustituimos los valores de df["spoken_languages"] por los valores de df_nuevo["nueva_col"]

In [10]:
# Chequeo si hay nulos o series con NaN en ellas y reemplazamos
df["revenue"].isnull().sum()
df["revenue"].fillna(0, inplace = True)

In [11]:
# dropeo las series con NaN
df = df.dropna(subset=["release_date"])

In [12]:
# Importo locale para cambiar las fechas y horarios a español
locale.setlocale(locale.LC_TIME, "es_ES")

'es_ES'

In [13]:
# Corrijo el el formato de la fecha
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

In [14]:
# Creo 3 nuevas columnas para trabajar mas facil con ellas
df["release_year"] = df["release_date"].dt.strftime('%Y')

df["release_month"] = df["release_date"].dt.strftime('%B')

df["release_day"] = df["release_date"].dt.strftime('%A')

In [15]:
df = df.dropna(subset=["release_date"])

In [16]:
# Calculo el retorno de las peliculas
df["budget"] = df["budget"].astype(float)
df["return"] = df["revenue"] / df["budget"]

In [17]:
# reemplazo donde me haya dado infinito o NaN por "0"
df["return"] = df["return"].replace([np.inf, 0])
df["return"] = df["return"].replace([np.NaN, 0])

Desanido las columnas del dataset "credits.csv"

In [18]:
df2 = pd.read_csv("credits.csv")

In [19]:
# Reemplazamos NaN por una lista compuesta por un diccionario vacío
df2['cast'] = df2['cast'].apply(lambda x: [] if pd.isna(x) else x)

# Utilizamos una función personalizada que verifique si el valor es una cadena y 
# luego intente convertirla en una lista para finalmente extraer los nombres del cast

def extract_cast_names(value):
    if isinstance(value, str):
        try:
            cast_list = ast.literal_eval(value)
            return [d.get('name') for d in cast_list]
        except (ValueError, SyntaxError):
            return None
    elif isinstance(value, list):
        return [d.get('name') for d in value]
    else:
        return None

#Creamos la columna "cast_names" compuesta por los valores de la clave
#"name" de los diccionarios de las listas de la columna "cast"

df2['cast_names'] = df2['cast'].apply(extract_cast_names)

#Reemplazamos las listas vacías por None
df2['cast_names'] = df2['cast_names'].replace([], None)

#Eliminamos la columna "cast" original
df2 = df2.drop('cast', axis=1)

In [20]:
# Reemplazamos NaN por una lista compuesta por un diccionario vacío
df2['crew'] = df2['crew'].apply(lambda x: [] if pd.isna(x) else x)

# Utilizamos una función personalizada que verifique si el valor es una cadena y 
#luego intente convertirla en una lista para finalmente extraer los nombres del crew

def extract_crew_names(value):
    if isinstance(value, str):
        try:
            crew_list = ast.literal_eval(value)
            return [d.get('name') for d in crew_list]
        except (ValueError, SyntaxError):
            return None
    elif isinstance(value, list):
        return [d.get('name') for d in value]
    else:
        return None

#Creamos la columna "crew_names" compuesta por los valores de la clave
#"name" de los diccionarios de las listas de la columna "crew"

df2['crew_names'] = df2['crew'].apply(extract_crew_names)

#Reemplazamos las listas vacías por None
df2['crew_names'] = df2['crew_names'].replace([], None)

#Eliminamos la columna "crew" original
df2 = df2.drop('crew', axis=1)

In [21]:
# Quito uno de las 2 columnas "id" para luego unir ambos dataframes
df2 = df2.drop("id", axis = 1)

In [22]:
# Uno ambos dataframes en uno para empezar a hacer las funciones
df = pd.concat([df ,df2], axis= 1)

Ahora construire las funciones que se requieren para hacer las peticiones a la API

In [23]:
def cantidad_filmaciones_mes(mes):
    '''Determina la cantidad de películas que se estrenaron en el mes dado'''
   
    peliculas_mes = df['id'][df['release_month'] == mes].count()
    return f"{int(peliculas_mes)} es la cantidad de películas que fueron estrenadas en el mes de {mes}"

In [24]:
def cantidad_filmaciones_dia(dia):
    '''Determina la cantidad de películas que se estrenaron en el día dado'''
   
    peliculas_dia = df['id'][df['release_day'] == dia].count()
    return f"{int(peliculas_dia)} es la cantidad de películas que fueron estrenadas en los días {dia}"


In [25]:
def score_titulo(titulo_de_la_filmacion):
    '''Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.'''

    titulo = titulo_de_la_filmacion
    anio_estreno = df["release_year"][df["title"] == titulo_de_la_filmacion]
    score = df["popularity"][df["title"] == titulo_de_la_filmacion]

    return f"La película {str(titulo)} fue estrenada en el año {int(anio_estreno)} con un score/popularidad de {float(score)}"

In [26]:
def votos_titulo(titulo_de_la_filmacion):
    '''Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.'''

    titulo = titulo_de_la_filmacion
    anio_estreno = int(df["release_year"][df["title"] == titulo_de_la_filmacion])
    cantidad_votos = int(df["vote_count"][df["title"] == titulo_de_la_filmacion])
    votos_promedio = float(df["vote_average"][df["title"] == titulo_de_la_filmacion])

    if cantidad_votos < 2000:
        return "Esta peliculas no cumple con la valoracion necesaria para una opinion objetiva"

    return f"La película {titulo} fue estrenada en el año {anio_estreno}. La misma cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {votos_promedio}"

In [27]:
def get_actor(nombre_actor):
    
    '''Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.'''

    contador = 0
    ganancia = 0
    retorno = 0

    for i in range(df["cast_names"].size):
        if nombre_actor in df["cast_names"][i]:
            ganancia += df.iloc[i, 10]
            retorno += df.iloc[i, 20]
            contador += 1
            prom_retorno = retorno / contador


    return f"El actor {nombre_actor} ha participado de {contador} filmaciones, el mismo ha conseguido un retorno de {ganancia} con un promedio de {prom_retorno} por filmación"

In [28]:
def get_director(nombre_director): 
    '''Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.'''

    retorno = 0

    titulo_pelicula = ""
    fecha_lanzamiento = ""
    retorno_ind = 0
    costo = 0
    peliculas = []

    for i in range(df["crew_names"].size):
        if nombre_director in df["crew_names"][i]:
            retorno += df.iloc[i, 10]

            titulo_pelicula =  peliculas.append(df.iloc[i, 14])
            fecha_lanzamiento =  peliculas.append(df.iloc[i, 9])
            retorno_ind =  peliculas.append(df.iloc[i, 10])
            costo = peliculas.append(df.iloc[i, 1])

    return f"Para el Director {nombre_director} el retorno a traves de su carrera fue de {retorno}, y aca informacion sobre sus peliculas: \n {peliculas}"

Creacion del modelo de recomendacion con ML

In [48]:
def recomendacion(titulo):

    num_recommendations=5
    # Cargar los datos de películas
    movies = pd.read_csv("data_con_EDA.csv")

    # Crear una matriz TF-IDF a partir de los títulos de las películas
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(movies["title"].values.astype('U'))

    # Entrenar un modelo de clustering (K-Means)
    kmeans = KMeans(n_clusters=10, random_state=42)
    kmeans.fit(tfidf_matrix)

    # Encontrar el cluster más cercano al título de la película proporcionado
    movie_vector = vectorizer.transform([titulo])
    cluster_label = kmeans.predict(movie_vector)[0]

    # Filtrar las películas en el mismo cluster que la película de interés
    cluster_movies = movies[kmeans.labels_ == cluster_label]

    # Excluir la película de interés de las recomendaciones
    cluster_movies = cluster_movies[cluster_movies["title"] != titulo]

    # Ordenar las películas por su similitud al título de interés
    cluster_movies["similitud"] = kmeans.transform(vectorizer.transform(cluster_movies["title"].values.astype('U'))).sum(axis=1)
    cluster_movies = cluster_movies.sort_values("similitud", ascending=False).head(num_recommendations)

    # Devolver las películas recomendadas
    recommendations = cluster_movies["title"].tolist()
    return recommendations

In [37]:
df.to_csv("data_con_EDA.csv", index=False)